# Text - Classification

Dataset : https://data.go.th/en/dataset/34 

Reference: https://huggingface.co/docs/transformers/tasks/sequence_classification

In [ ]:
!pip install datasets
!pip install pandas
!pip install pandas-profiling
!pip install transformers
!pip install sentencepiece
!pip install -U scikit-learn

## Preparing Data

> Download file.csv มาเป็น dataframe แล้วแสดงข้อมูลโดยใช้ library `pandas`

In [ ]:
from datasets import load_dataset
import pandas as pd

df = pd.read_csv("./noit11561118811.csv") # load the dataset
df.head()

> เลือกเฉพาะ column ที่จำเป็นต่อการใช้งานในนี้คือ `PR_PROD_NAME` และ `PRICE_DAY` โดยเปลี่ยนชื่อ column เป็น `text` และ `price` ตามลำดับ

In [ ]:
df.drop(columns=["PRICE_DATE", "MARKET_NAME"], inplace=True) # drop column
df.rename(columns = {"PR_PROD_NAME":"text", "PRICE_DAY":"price"}, inplace = True) # rename column
df.dropna(inplace=True) # drop null values
df.head() # show first 5 rows

In [ ]:
df["price"].describe() # show statistics

> สร้าง dataframe ใหม่ที่มีเฉพาะข้อมูลที่ต้องการ โดยให้สินค้าที่มี

ราคาต่ำกว่า 100 มี label = 0


สินค้าที่มีราคา 100-1000 มี label = 1


และมากว่า 1000 มี label = 2

In [ ]:
labels = []
for price in df["price"]:
    if  price < 100:
        labels.append(0) # 0 - 100 = Cheap Price
    elif price <= 1000:
        labels.append(1) # 100 - 1000 = Normal Price
    else:
        labels.append(2) # 1000 +++  =  Expensive Price

df["label"] = labels
df.sample(frac=1)

 drop `column price` ที่ไม่ได้ใช้งาน

In [ ]:
df.drop(columns=["price"], inplace=True) # drop column
df.head() # show first 5 rows

> แบ่งข้อมูลเพื่อใช้ train 80% และ test 20%

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.to_csv("./train.csv", index=False)
test_df.to_csv("./test.csv", index=False)

library `pandas_profiling` สามารถช่วย visualize data เพื่อเช็คข้อมูลเบื้องต้นของ data ได้

In [ ]:
# Visualize data
import ydata_profiling as pp
pp.ProfileReport(df)

## Preprocessing

ก่อนจะนำข้อมูลไปใช้ในการ train ต้องทำการเตรียมข้อมูลให้เหมาะสมก่อน โดยใช้ library `transformers` ในการทำ `tokenize` และ `encode` ข้อมูล

In [ ]:
dataset = load_dataset("csv", data_files={"train": "train.csv", "test" : "test.csv" })
dataset

ใช้ `wangchanberta` tokenizer เพราะ dataset ที่เป็นภาษาไทย

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

# load tokenizer from pretrained model which alrady has a vocabulary
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", use_fast=False)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_imdb = dataset.map(preprocess_function, batched=True) # tokenize dataset

print(tokenized_imdb["train"][0]) # print first row of the tokenized training set

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # set format

## Training Model

หลังจากที่เตรียมข้อมูลเสร็จแล้ว ก็นำข้อมูลไป train โดยใช้ `Trainer` และ `TrainingArguments`

In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# define accuracy metric
accuracy = evaluate.load("accuracy")

# กำหนด label ให้กับข้อมูล
id2label = {0: "Low Price", 1: "Normal Price", 2: "High Price"}
label2id = {"Low Price": 0, "Normal Price": 1, "High Price": 2}

ทำการเลือก model ที่จะนำมาใช้ในการ train ก่อน ในที่นี้ใช้ `wangchanberta-base` ซึ่งเป็น model ที่มีความเร็วในการ train และให้ความแม่นยำสูง

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "airesearch/wangchanberta-base-att-spm-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

กำหนด `TrainingArguments` โดยกำหนด `parameters` ต่างๆในการเทรน model

In [ ]:
training_args = TrainingArguments(
    output_dir="PriceClassification", # โฟลเดอร์เก็บไฟล์ model ที่ train แล้ว
    learning_rate=0.00001, # อัตราการเรียนรู้
    per_device_train_batch_size=16, # จำนวนข้อมูลที่จะนำมา train ในแต่ละครั้ง
    per_device_eval_batch_size=16, # จำนวนข้อมูลที่จะนำมา test ในแต่ละครั้ง
    num_train_epochs=2, # จำนวนรอบในการ train 
    weight_decay=0.01, # ค่าความถี่ในการปรับค่า weight
    evaluation_strategy="epoch", # กำหนดการทดสอบ model ทุกๆ 1 epoch
    save_strategy="epoch", # กำหนดการบันทึก model ทุกๆ 1 epoch
    load_best_model_at_end=True, # โหลด model ที่ดีที่สุดมาใช้
    push_to_hub=False, 
    report_to="none"
)

trainer = Trainer( 
    model=model, 
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics, 
)

In [ ]:
trainer.train() # train model

## Inference

หลังจากทำการเทรน model แล้ว เราสามารถนำ model ที่ได้ไปใช้งานได้ โดยใช้ `pipeline` ในการทำนายผล

In [ ]:
# text = "ยางพาราแผ่นดิบ" # Low Price
text = "มะพร้าวผลแห้งทั้งเปลือก" # Normal Price
# text = "ข้าวเปลือกเจ้านาปีพันธุ์ขาวดอกมะลิ105ชนิด" # High Price

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="PriceClassification/checkpoint-894/") # load model ตามที่ได้เทรนไว้
classifier(text)

> optionnal 

สามารถทำการ load model โดยใช้ `from_pretrained` และทำนายผลในอีกวิธี

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
from transformers import AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("PriceClassification/checkpoint-2235/", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", use_fast=False)
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]